In [6]:
import os
from PIL import Image
from array import *
from random import shuffle

import shutil
import glob

if not os.path.exists('cgan_imgs/train/'):
    os.makedirs('cgan_imgs/train/')

if not os.path.exists('cgan_imgs/test/'):
    os.makedirs('cgan_imgs/test/')


In [7]:
# Now divide the dataset to 80% train and 20% test data 
data_path = 'cgan_imgs'
classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

for folder_name in classes:
    train_path = str(data_path + '/train/' +folder_name+'/')
    if not os.path.exists(train_path):
        os.makedirs(train_path)
    test_path = str(data_path + '/test/' +folder_name+'/')
    if not os.path.exists(test_path):
        os.makedirs(test_path)
    
    source_path = os.path.join(data_path, folder_name, '*.jpg')

    files = glob.glob(source_path)
    
    for fl in files[:1200]:
        shutil.move(fl, train_path)

    for fl in files[1200:]:
        shutil.move(fl, test_path)

In [8]:

# Load from and save to
Names = [['cgan_imgs/train','train'], ['cgan_imgs/test','test']]

for name in Names:
    data_image = array('B')
    data_label = array('B')

    FileList = []
    for dirname in os.listdir(name[0])[1:]: # [1:] Excludes .DS_Store from Mac OS
        path = os.path.join(name[0],dirname)
        for filename in os.listdir(path):
            if filename.endswith(".png"):
                FileList.append(os.path.join(name[0],dirname,filename))

    shuffle(FileList) # Usefull for further segmenting the validation set

    for filename in FileList:

        label = int(filename.split('/')[2])

        Im = Image.open(filename)

        pixel = Im.load()

        width, height = Im.size

        for x in range(0,width):
            for y in range(0,height):
                data_image.append(pixel[y,x])

        data_label.append(label) # labels start (one unsigned byte each)

    hexval = "{0:#0{1}x}".format(len(FileList),6) # number of files in HEX

    # header for label array

    header = array('B')
    header.extend([0,0,8,1,0,0])
    header.append(int('0x'+hexval[2:][:2],16))
    header.append(int('0x'+hexval[2:][2:],16))

    data_label = header + data_label

    # additional header for images array
    
    if max([width,height]) <= 256:
        header.extend([0,0,0,width,0,0,0,height])
    else:
        raise ValueError('Image exceeds maximum size: 256x256 pixels');

    header[3] = 3 # Changing MSB for image data (0x00000803)

    data_image = header + data_image

    output_file = open(name[1]+'-images-idx3-ubyte', 'wb')
    data_image.tofile(output_file)
    output_file.close()

    output_file = open(name[1]+'-labels-idx1-ubyte', 'wb')
    data_label.tofile(output_file)
    output_file.close()

# gzip resulting files

for name in Names:
    os.system('gzip '+name[1]+'-images-idx3-ubyte')
    os.system('gzip '+name[1]+'-labels-idx1-ubyte')
    

NameError: name 'width' is not defined